# 04_무신사 반팔 데이터

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [2]:
# ChromeDriver 설치 및 경로 설정
service = Service(ChromeDriverManager().install())

In [3]:
# 옵션 설정
options = Options()
options.add_experimental_option("detach", True)  # 브라우저 자동 종료 방지
options.add_experimental_option("excludeSwitches", ["enable-logging"])  # 로그 제거

In [4]:
# 페이지 오픈
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.musinsa.com/search/goods?keyword=%EB%B0%98%ED%8C%94&keywordType=popular&gf=A")
time.sleep(5)

In [5]:
# 스크롤 (더 많은 상품 로딩)
scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print("[INFO] 스크롤 완료, 데이터 수집 시작")

# 상품 컨테이너 찾기
items = driver.find_elements(By.CLASS_NAME, "sc-d36st-1")
print(f"[INFO] 총 {len(items)}개의 상품을 찾았습니다.")

data = []
for idx, item in enumerate(items, start=1):
    try:
        # 브랜드명
        brand = item.find_element(By.CSS_SELECTOR, "a[aria-label*='샵으로 이동']").text.strip()
        # 상품명
        product_name = item.find_element(By.CSS_SELECTOR, "a[aria-label*='상품상세로 이동'] span").text.strip()
        # 가격
        price = item.find_element(By.CSS_SELECTOR, "span[data-mds='Typography']").text.strip()
        # 상품 링크
        product_link = item.find_element(By.CSS_SELECTOR, "a[aria-label*='상품 상세로 이동']").get_attribute("href")
        # 이미지 링크
        image_link = item.find_element(By.TAG_NAME, "img").get_attribute("src")

        # 좋아요 수 (없을 수도 있음)
        try:
            like_count = item.find_element(By.CSS_SELECTOR, "span.text-red").text.strip()
        except:
            like_count = ""

        # 평점 (없을 수도 있음)
        try:
            rating = item.find_element(By.CSS_SELECTOR, "span.text-yellow").text.strip()
        except:
            rating = ""

        data.append({
            "브랜드": brand,
            "상품명": product_name,
            "가격": price,
            "좋아요": like_count,
            "평점": rating,
            "상품 링크": product_link,
            "이미지 링크": image_link
        })

        print(f"[{idx}] {brand} | {product_name} | {price}")
        
    except Exception as e:
        print(f"[ERROR] {idx}번째 아이템 처리 중 오류: {e}")

[INFO] 스크롤 완료, 데이터 수집 시작
[INFO] 총 30개의 상품을 찾았습니다.
[1] 오퍼스 0012 | Op. 07 IXO T-SHIRTS WHITE | 멤버스데이
[2] 지프 | [190g] M-Logo 그래픽 반팔 티 (JR5TSU173WH) | 멤버스데이
[3] 오언더알 | RIDING BIKE CAT 오버핏 반팔 티셔츠 (화이트) | 멤버스데이
[4] 리 | 스몰 트위치 로고 2PACK 티셔츠 컬러 믹스 | 멤버스데이
[5] 로긴 앤 로지 | Ribbon Printed Slim Ringer T-shirt Ivory | 멤버스데이
[ERROR] 6번째 아이템 처리 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[aria-label*='샵으로 이동']"}
  (Session info: chrome=138.0.7204.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
0   chromedriver                        0x00000001031b8b38 cxxbridge1$str$ptr + 2722088
1   chromedriver                        0x00000001031b0aa8 cxxbridge1$str$ptr + 2689176
2   chromedriver                        0x0000000102d0233c cxxbridge1$string$len + 90648
3   chromedriver                        0x0000000102d49494 cxxbridge1$string$len + 381808


In [6]:
df = pd.DataFrame(data)
df.to_csv("data/무신사_반팔.csv", index=False, encoding="utf-8-sig")

In [7]:
# 종료
driver.quit()